In [1]:
from gensim import corpora, models
import pandas as pd
import numpy as np
import sys
from collections import Counter
import itertools
from lsi_tagger.model import TagExtractor

%load_ext autoreload
%autoreload 2

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\David\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [2]:
df = pd.read_csv('data/h&m_kaggle_products.csv')
df = df[~df.isnull().any(axis=1)]
print(df.shape)
df.head(2)

(105126, 25)


,article_id,product_code,prod_name,product_type_no,product_type_name,product_group_name,graphical_appearance_no,graphical_appearance_name,colour_group_code,colour_group_name,...,department_name,index_code,index_name,index_group_no,index_group_name,section_no,section_name,garment_group_no,garment_group_name,detail_desc
0,108775015,108775,Strap top,253,Vest top,Garment Upper body,1010016,Solid,9,Black,...,Jersey Basic,A,Ladieswear,1,Ladieswear,16,Womens Everyday Basics,1002,Jersey Basic,Jersey top with narrow shoulder straps.
1,108775044,108775,Strap top,253,Vest top,Garment Upper body,1010016,Solid,10,White,...,Jersey Basic,A,Ladieswear,1,Ladieswear,16,Womens Everyday Basics,1002,Jersey Basic,Jersey top with narrow shoulder straps.


In [3]:
documents = df['detail_desc'].drop_duplicates().values

te = TagExtractor(word_count_min=2, word_length_min=2, num_lsi_topics=200, 
                  bigram_kwargs={'bigrams_pmi_min_value':1, 'bigrams_min_freq':200})
te.fit(documents)

Filtering tokens: 100%|████████████████████████████████████████████████████████| 43404/43404 [00:12<00:00, 3423.60it/s]
Making bigrams: 43404it [00:00, 52742.43it/s]
Filtering by word count: 100%|████████████████████████████████████████████████| 43404/43404 [00:00<00:00, 66895.07it/s]
2022-04-09 19:56:43 INFO     Training TF-IDF...
2022-04-09 19:56:43 INFO     adding document #0 to Dictionary(0 unique tokens: [])
2022-04-09 19:56:43 INFO     adding document #10000 to Dictionary(1742 unique tokens: ['jersey', 'jersey top', 'narrow', 'narrow shoulder', 'shoulder']...)
2022-04-09 19:56:44 INFO     adding document #20000 to Dictionary(1938 unique tokens: ['jersey', 'jersey top', 'narrow', 'narrow shoulder', 'shoulder']...)
2022-04-09 19:56:45 INFO     adding document #30000 to Dictionary(2070 unique tokens: ['jersey', 'jersey top', 'narrow', 'narrow shoulder', 'shoulder']...)
2022-04-09 19:56:45 INFO     adding document #40000 to Dictionary(2181 unique tokens: ['jersey', 'jersey top', 'nar

In [4]:
len(te.tc.bigrams_dict)

213

In [5]:
input_tags, candidate_tags = te.transform(input_document='Jersey top with narrow shoulder straps.',
                                          candidate_documents=[
                                              'Tights with built-in support to lift the bottom. Black in 30 denier and light amber in 15 denier.',
                                              'Semi shiny tights that shape the tummy, thighs and calves while also encouraging blood circulation in the legs. Elasticated waist.',
                                              'Jersey top with narrow shoulder straps.',
                                              'uygnhjkl'
                                          ], 
                                          n_input_tags=10, n_candidate_tags=10)

input_tags, candidate_tags

([('jersey top', 1),
  ('narrow shoulder', 1),
  ('shoulder straps', 1),
  ('narrow', 1),
  ('straps', 1),
  ('shoulder', 1),
  ('top', 1),
  ('jersey', 1)],
 [{},
  {},
  {'jersey top': 0.8540828783737447,
   'narrow shoulder': 0.8076031140479123,
   'shoulder straps': 0.6303037652655175,
   'narrow': 0.6078363836697849,
   'straps': 0.587746604876064,
   'shoulder': 0.5606345662980713,
   'top': 0.3652303599135097,
   'jersey': 0.36017823388046155},
  {}])

In [6]:
new_candidate_ranking = te.rank(candidate_tags=candidate_tags, selected_tags=['jersey'])
new_candidate_ranking

[2, 0, 1, 3]

In [7]:
te.save()
te = TagExtractor()
te.load()

In [8]:
input_tags, candidate_tags = te.transform(input_document='Jersey top with narrow shoulder straps.',
                                          candidate_documents=[
                                              'Tights with built-in support to lift the bottom. Black in 30 denier and light amber in 15 denier.',
                                              'Semi shiny tights that shape the tummy, thighs and calves while also encouraging blood circulation in the legs. Elasticated waist.',
                                              'Jersey top with narrow shoulder straps.',
                                              'uygnhjkl'
                                          ], 
                                          n_input_tags=10, n_candidate_tags=10)

input_tags, candidate_tags

([('jersey top', 1),
  ('narrow shoulder', 1),
  ('shoulder straps', 1),
  ('straps', 1),
  ('narrow', 1),
  ('shoulder', 1),
  ('top', 1),
  ('jersey', 1)],
 [{},
  {},
  {'jersey top': 0.8772317763823081,
   'narrow shoulder': 0.7785568909779876,
   'shoulder straps': 0.6355535130824466,
   'straps': 0.6059983392044233,
   'narrow': 0.6053735593033779,
   'shoulder': 0.5555633523654915,
   'top': 0.36434324889827213,
   'jersey': 0.35461624727365776},
  {}])

In [9]:
new_candidate_ranking = te.rank(candidate_tags=candidate_tags, selected_tags=['jersey'])
new_candidate_ranking

[2, 0, 1, 3]

In [14]:
# TODO: Can give the user options of adjacent tags so that they can get new queries entirely?
# They can remove tags from the input document and add in other tags, like swapping out pants for jeans, etc.
# This results in a "new" input document

lsi_topic_matrix_T_normed = te.lsi_topic_matrix.T/np.linalg.norm(te.lsi_topic_matrix.T, axis=1)[:,None]

def lsi_corpus2vec(lsi_corpus_vec):
    vec = np.zeros(200)
    for ind, value in lsi_corpus_vec:
        vec[ind] = value
    return vec

tags = te.tc.transform([df['detail_desc'].drop_duplicates().values[777]])[0]
print(tags)
bow = te.dictionary.doc2bow(tags)
bow_vec = lsi_corpus2vec(te.lsi_model[te.tfidf[bow]])
bow_vec_normed = bow_vec/np.linalg.norm(bow_vec)
sims = bow_vec_normed.dot(lsi_topic_matrix_T_normed.T)
top_inds = np.argsort(-sims)
count = 0
for t in top_inds:
    tag = te.dictionary[t]
    if tag not in tags:
        print(tag)
        count += 1
    if count == 5:
        break

['single', 'breasted', 'jacket', 'stretch', 'weave', 'containing', 'wool', 'narrow', 'notch', 'lapels', 'decorative', 'buttonhole', 'fastening', 'buttons', 'chest', 'pocket', 'flap', 'pockets', 'inner', 'pockets', 'button', 'decorative', 'buttons', 'cuffs', 'single', 'vent', 'lined', 'skinny', 'fit', 'slightly', 'shorter', 'style', 'shapes', 'chest', 'tapers', 'sharply', 'waist', 'combined', 'narrow', 'shoulders', 'sleeves', 'creates', 'completely', 'fitted', 'silhouette']
narrower
tuxedo
special
linked
search
